In [10]:
import cv2
import open3d as o3d
import numpy as np
import colorsys
import pyransac3d as pyrsc
import copy

In [11]:
def visualizer(cloud):

    if type(cloud) == o3d.cpu.pybind.geometry.TriangleMesh:
        vertices = np.asarray(cloud.vertices)
        center_of_mass = np.mean(vertices, axis=0)
    else:
        # point cloud
        center_of_mass = np.mean(np.asarray(cloud.points), axis=0)
    
    # print("com", center_of_mass)
    o3d.visualization.draw_geometries([cloud],
                                  zoom=0.5,
                                  front=[1, 0, 0],
                                  lookat=center_of_mass,
                                  up=[0, 0, 1])


In [12]:
pcd = o3d.io.read_point_cloud("3dmodels/Strawberry_New.ply")

visualizer(pcd)

In [13]:
# -2 to 2
bounding_box = o3d.geometry.AxisAlignedBoundingBox(min_bound=(-1, -1, -1),
                                                    max_bound=(1, 1, 1))

filtered_cloud = pcd.crop(bounding_box)

visualizer(filtered_cloud)

In [14]:
threshold = 60 # in % max, min: [1.9655249  2.17742805 1.19400877], [-1.96551809e+00 -2.17745111e+00  9.51918373e-04]
center_pc = pcd.get_center()
max_v = pcd.get_max_bound()
min_v = pcd.get_min_bound()

size = (max_v - min_v) * 0.01 * threshold

min_bound = center_pc - size
max_bound = center_pc + size
bounding_box = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound,
                                                    max_bound=max_bound)

filtered_cloud = pcd.crop(bounding_box)

visualizer(filtered_cloud)

In [15]:
s = 3  #no. of samples used every time to create the shape
p = 0.99 #99%
e = 0.5 # 50%
k = np.log(1-p)/(np.log(1 - np.power((1-e), s)))
print(k)
# k => number of iterations

34.48754705148163


In [16]:

# inlier_cloud = filtered_point_cloud.select_by_index(best_inliers_sphere)
# outlier_cloud = filtered_point_cloud.select_by_index(best_inliers_sphere, invert=True)

# # outlier_cloud.paint_uniform_color([1, 0, 0])
# # inlier_cloud.paint_uniform_color([0, 1, 0])
# o3d.visualization.draw_geometries([outlier_cloud],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [17]:
def cloud_based_point_index(indexes, inlier=False):
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0, 1, 0])
    if not inlier:
        outlier_cloud = filtered_cloud.select_by_index(indexes, invert=True)

        return outlier_cloud

    else:
        inlier_cloud = filtered_cloud.select_by_index(indexes)
        o3d.visualization.draw_geometries([inlier_cloud],
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])
        
        return inlier_cloud


In [18]:
point = pyrsc.Point()  #Using RANSAC
# s = 1
center, best_inliers_point = point.fit(np.asarray(filtered_cloud.points), thresh=0.7, maxIteration=10)

inlier_cloud_point = filtered_cloud.select_by_index(best_inliers_point)
outlier_cloud_point = filtered_cloud.select_by_index(best_inliers_point, invert=True)

o3d.visualization.draw_geometries([inlier_cloud_point],
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])

In [19]:
# sphere = pyrsc.Sphere()
# # s = 4
# center, radius, best_inliers_sphere = sphere.fit(np.asarray(inlier_cloud_point.points), thresh=0.1, maxIteration=75)

# inlier_cloud_sphere = inlier_cloud_point.select_by_index(best_inliers_sphere)
# outlier_cloud_sphere = inlier_cloud_point.select_by_index(best_inliers_sphere, invert=True)

# # outlier_cloud_sphere.paint_uniform_color([1, 0, 0])
# # inlier_cloud_sphere.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([outlier_cloud_sphere],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [20]:
# # plane??

# sphere = pyrsc.Plane()
# # s = 3
# equation, best_inliers_plane = sphere.fit(np.asarray(inlier_cloud_point.points), thresh=0.095, maxIteration=40)

# inlier_cloud_sphere = inlier_cloud_point.select_by_index(best_inliers_plane)
# outlier_cloud_sphere = inlier_cloud_point.select_by_index(best_inliers_plane, invert=True)

# # outlier_cloud_sphere.paint_uniform_color([1, 0, 0])
# # inlier_cloud_sphere.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([outlier_cloud_sphere],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [21]:
# print('octree division')
# octree = o3d.geometry.Octree(max_depth=15)
# octree.convert_from_point_cloud(outlier_cloud_sphere, size_expand=0.0001)
# o3d.visualization.draw_geometries([octree])

In [22]:
pcd = inlier_cloud_point

In [23]:
def convert_rgb_2_hsv(all_rgb_colors):
    all_hsv_colors = []

    for i in range(len(all_rgb_colors)):
        temp_color = all_rgb_colors[i]
        temp = colorsys.rgb_to_hsv(temp_color[0], temp_color[1], temp_color[2])
        all_hsv_colors.append(temp)

    all_hsv_colors = np.asarray(all_hsv_colors)

    return all_hsv_colors

all_rgb_colors = np.asarray(pcd.colors)
all_hsv_colors = convert_rgb_2_hsv(all_rgb_colors)


In [24]:
def hsv_filter(hsv_color):
    low_h = 2/360
    high_h = 180/360

    low_s = 0/255
    high_s = 40/255

    low_v = 0/255
    high_v = 255/255

    # removes the background

    if low_h <= hsv_color[0] <= high_h and low_s <= hsv_color[1] <= high_s and low_v <= hsv_color[2] <= high_v:
        return True
    else:
        return False
    

In [25]:
def hsv_filter_red(hsv_color):

    # print("color:", hsv_color)
    low_h = 0/360
    high_h = 40/360 #20

    low_h_1 = 340/360
    high_h_1 = 360/360

    # 330 to 30 degrees

    low_s = 0.1 #0.3
    high_s = 1

    low_v = 0.4
    high_v = 1

    # red
    # Lower bound: (0, 100, 100)
    # Upper bound: (10, 255, 255)

    if ((low_h <= hsv_color[0] <= high_h) or (low_h_1 <= hsv_color[0] <= high_h_1))and low_s <= hsv_color[1] <= high_s and low_v <= hsv_color[2] <= high_v:
        return True
    else:
        return False
    

In [26]:
new_points = []
new_colors = []
new_colors_RGB_u = []

for index, new_color in enumerate(all_hsv_colors):
    new_point = pcd.points[index]
    new_color_RGB = all_rgb_colors[index]
    # if hsv_filter(new_color):
    #     new_colors_RGB_u.append(new_color_RGB)
    #     new_points.append(new_point)
    #     new_colors.append(new_color)
    #     # print(new_color_RGB)
    if hsv_filter_red(new_color):
        # break
        new_colors_RGB_u.append(new_color_RGB)
        new_points.append(new_point)
        new_colors.append(new_color)

new_colors_RGB_u = np.asarray(new_colors_RGB_u)
new_colors = np.asarray(new_colors)
new_points = np.asarray(new_points)

filtered_point_cloud = o3d.geometry.PointCloud()
filtered_point_cloud.points = o3d.utility.Vector3dVector(new_points)
filtered_point_cloud.colors = o3d.utility.Vector3dVector(new_colors_RGB_u)
visualizer(filtered_point_cloud)

In [ ]:
cl, ind = filtered_point_cloud.remove_statistical_outlier(nb_neighbors=70,
                                                    std_ratio=1.0)
cl.estimate_normals()
visualizer(cl)


com [ 0.14531587 -0.39028612  0.26856929]


In [ ]:
# plane??

sphere = pyrsc.Plane()
# s = 3
equation, best_inliers_plane = sphere.fit(np.asarray(cl.points), thresh=0.095, maxIteration=40)

inlier_cloud_plane = cl.select_by_index(best_inliers_plane)
outlier_cloud_plane = cl.select_by_index(best_inliers_plane, invert=True)

# outlier_cloud_plane.paint_uniform_color([1, 0, 0])
# inlier_cloud_plane.paint_uniform_color([0, 1, 0])

o3d.visualization.draw_geometries([outlier_cloud_plane],
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])

In [ ]:
filtered_point_cloud, _ = cl.remove_statistical_outlier(nb_neighbors=5, std_ratio=0.5)

visualizer(filtered_point_cloud)

com [ 0.1511416  -0.36989658  0.27466337]


In [ ]:
filtered_point_cloud_r, _ = filtered_point_cloud.remove_radius_outlier(nb_points=15, radius=0.01)

visualizer(filtered_point_cloud_r)


com [ 0.20981086 -0.23630501  0.33284062]


In [ ]:
filtered_point_cloud_r = filtered_point_cloud

In [ ]:
# Kmeans to segment
from sklearn.cluster import KMeans

# filtered_point_cloud = o3d.geometry.PointCloud()
# def filter_red(points, colors):
# Convert RGB colors to feature space suitable for clustering
features = np.array(filtered_point_cloud_r.points)  # Use RGB directly as features

# Apply K-means clustering
kmeans = KMeans(n_clusters=19, random_state=0).fit(features)

# Identify cluster representing red points
# kmeans.cluster_centers_ -> gives the cenetrs of the clusters so i.e of the colors
red_cluster_index = np.argmin(np.mean(kmeans.cluster_centers_, axis=1))

# Filter points based on cluster membership
# red_points = np.asarray(pcd.points)[kmeans.labels_ == red_cluster_index]
# red_colors = np.asarray(pcd.colors)[kmeans.labels_ == red_cluster_index]

# return red_points, red_colors

# filtered_point_cloud.points = o3d.utility.Vector3dVector(red_points)
# filtered_point_cloud.colors = o3d.utility.Vector3dVector(red_colors)

color = [
    [255, 0, 0],   # Red
    [0, 255, 0],   # Green
    [0, 0, 255],   # Blue
    [255, 255, 0], # Yellow
    [255, 0, 255], # Magenta
    [0, 255, 255], # Cyan
    [0, 128, 0],   # Green (dark)
    [0, 0, 128],   # Navy
    [128, 128, 0],  # Olive
    [128, 0, 128],  # Purple
    [0, 128, 128],  # Teal
    [128, 128, 128],# Gray
    [255, 165, 0],  # Orange
    [255, 192, 203],# Pink
    [0, 0, 0],      # Black
    [255, 255, 255],# White
    [128, 0, 0],    # Maroon (dark)
    [0, 128, 0],    # Green (medium)
    [0, 0, 128]  
]

outlier_cloud_sph = []
final_segments  = []

avg = []

for i in range(len(kmeans.cluster_centers_)):
    temp_p = np.where(kmeans.labels_ == i)
    temp_sphere = filtered_point_cloud_r.select_by_index(temp_p[0])
    temp_points = np.asarray(filtered_point_cloud_r.colors)[kmeans.labels_ == i]
    temp_avg = np.average(temp_points, axis=0)
    avg.append(temp_avg)
    deep_copy_seg = copy.deepcopy(temp_sphere)
    final_segments.append(deep_copy_seg)
    outlier_cloud_sph.append(temp_sphere)
    temp_sphere.paint_uniform_color(color[i])

geometries_to_visualize = [outlier_cloud_sph[i] for i in range(len(kmeans.cluster_centers_))]  # Adjust the range as needed

o3d.visualization.draw_geometries(geometries_to_visualize,
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])



[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to

In [ ]:
def convert_rgb_2_hsv(all_rgb_colors):
    all_hsv_colors_segement = []

    for i in range(len(all_rgb_colors)):
        temp_color = all_rgb_colors[i]
        temp = colorsys.rgb_to_hsv(temp_color[0], temp_color[1], temp_color[2])
        all_hsv_colors_segement.append(temp)

    all_hsv_colors_segement = np.asarray(all_hsv_colors_segement)

    return all_hsv_colors_segement

all_hsv_colors_segement = convert_rgb_2_hsv(avg)

print(all_hsv_colors_segement)

index = np.argmin(all_hsv_colors_segement, axis=0)[0]

visualizer(final_segments[index])

[[0.09074702 0.35829127 0.64542986]
 [0.09430054 0.20623786 0.62381553]
 [0.08650224 0.42516879 0.53192749]
 [0.09520591 0.27250626 0.5218762 ]
 [0.09722635 0.16132125 0.55646259]
 [0.09160605 0.53674782 0.55526861]
 [0.08461302 0.31069929 0.54098815]
 [0.09601218 0.32564094 0.55290292]
 [0.08935027 0.1894224  0.57592968]
 [0.08685121 0.34048068 0.55272412]
 [0.02403794 0.62934027 0.68546075]
 [0.07708701 0.3048965  0.56417047]
 [0.09701686 0.19978234 0.58432887]
 [0.09154102 0.25120512 0.54983097]
 [0.07292691 0.37504305 0.54692093]
 [0.10088496 0.1761588  0.69876543]
 [0.06571062 0.2247883  0.63676471]
 [0.09878158 0.36578249 0.69473247]
 [0.09622021 0.20039519 0.6843588 ]]
com [0.22423968 0.05709186 0.32194959]


In [ ]:
cld = final_segments[index]

In [ ]:
sphere = pyrsc.Sphere()
# s = 4
center, radius, best_inliers_sphere = sphere.fit(np.asarray(cld.points), thresh=0.02, maxIteration=75)

inlier_cloud_sphere_new = cld.select_by_index(best_inliers_sphere)
# outlier_cloud_sphere = cld.select_by_index(best_inliers_sphere, invert=True)

# outlier_cloud_sphere.paint_uniform_color([1, 0, 0])
# inlier_cloud_sphere.paint_uniform_color([0, 1, 0])

visualizer(inlier_cloud_sphere_new)
# o3d.visualization.draw_geometries([outlier_cloud_sphere, inlier_cloud_sphere])

cld = inlier_cloud_sphere_new

com [0.22496017 0.06124204 0.33235699]


In [ ]:
# Get the center of the strawberry
center_strawberry = cld.get_center()

center_pc = o3d.geometry.PointCloud()
center_pc.points = o3d.utility.Vector3dVector(np.asarray([center_strawberry]))
center_pc.paint_uniform_color([0, 0, 1])

o3d.visualization.draw_geometries([cld, center_pc])

In [ ]:
radii = [0.005, 0.01]
cld.estimate_normals()
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
    cld, alpha=0.01)

# with o3d.utility.VerbosityContextManager(
#         o3d.utility.VerbosityLevel.Debug) as cm:
#     mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
#         cld, depth=10)

visualizer(mesh)

# Can make much better meshes


[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
com [0.22495377 0.06125265 0.33235703]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [ ]:
def color_bounding_box(box):
    # visualizer(mesh)
    vertices = np.asarray(box.get_box_points())

    # Define the edges of the oriented bounding box
    lines = [
        [0, 1], [1, 2], [2, 3], [3, 0],
        [4, 5], [5, 6], [6, 7], [7, 4],
        [0, 4], [1, 5], [2, 6], [3, 7]
    ]

    # Create a LineSet to represent the box
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(vertices),
        lines=o3d.utility.Vector2iVector(lines)
    )

    # Set the color of the box (e.g., red)
    line_set.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (len(lines), 1)))
    # o3d.visualization.draw_geometries([mesh, line_set],
    #                                 zoom=0.5,
    #                                 front=[1, 0, 0],
    #                                 lookat=[-0.05272632, -0.02440209, 0.48188474],
    #                                 up=[0, 0, 1])

    return line_set

In [ ]:
_, _, v = np.linalg.svd(np.asarray(cld.points))
principal_axes = v[:3, :]

# Create coordinate axes with the y-axis aligned along the principal axis with the largest eigenvalue
coordinate_axes = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1, origin=np.mean(np.asarray(cld.points), axis=0))

# Visualize the strawberry point cloud along with the coordinate axes
o3d.visualization.draw_geometries([cld, coordinate_axes],
                                  zoom=0.5,
                                  front=[1, 0, 0],
                                  lookat=np.mean(np.asarray(cld.points), axis=0),
                                  up=[0, 0, 1])

In [ ]:
# AABB box
box_triangle = mesh.get_axis_aligned_bounding_box()
line_set_triangle = color_bounding_box(box_triangle)
o3d.visualization.draw_geometries([mesh, line_set_triangle, center_pc])

NameError: name 'mesh' is not defined

In [ ]:
mesh = mesh.compute_triangle_normals()
vertices = np.asarray(mesh.vertices)
center_of_mass = np.mean(vertices, axis=0)
m1 = mesh.create_coordinate_frame(size=0.00001, origin=center_of_mass)
visualizer(m1)

com [0.22495429 0.06125317 0.33235754]


In [ ]:
hull, _ = cld.compute_convex_hull()  #cld -> is the point cloud
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((0, 0 , 1))
# visualizer(hull_ls)
o3d.visualization.draw_geometries([cld, hull])

In [ ]:
hull.remove_degenerate_triangles()
hull.compute_vertex_normals()
hull.orient_triangles()
o3d.visualization.draw_geometries([cld, hull])
volume = hull.get_volume()
print(volume)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
0.00018634931720575216


In [ ]:
volume #in mm^3??

0.000596328157324427

In [ ]:
# whole plant
hull_new, ls_new = pcd.compute_convex_hull()
hull_new.remove_degenerate_triangles()
hull_new.compute_vertex_normals()

o3d.visualization.draw_geometries([pcd, hull_new])
volume = hull_new.get_volume()

In [ ]:
volume

4.158153960855363

In [ ]:
# d = m/v
density = 980 	 #in g/m^3 #1146.43 #in kg/m^3
v_new = 5.963281573244269439e-13
mass = density*v_new
print(mass)

5.844015941779384e-10
